In [37]:
# Stores a matrix of the form M(v) = Diagonal(v) + vv' where v is a vector.
# Well, it only stores the vector v to conserve memory.
struct Custom
    v::Array{Float64,1}
end

In [38]:
function custom_matrix_str(v)
    size(v, 1) == 0 && return ""
    size(v, 1) == 1 && return string(v[1] + v[1]^2)
    
    s = ""
    for i in 1:size(v, 1)
        for j in 1:size(v, 1)
            if i == j
                s *= " $(v[i] + v[i]^2) \t"
            else
                s *= " $(v[i] * v[j]) \t"
            end
        end
        s *= '\n'
    end
    
    return s
end

Base.show(io::IO, M::Custom) = print(io, custom_matrix_str(M.v))

In [39]:
M = Custom([1,2,3,4])

 2.0 	 2.0 	 3.0 	 4.0 	
 2.0 	 6.0 	 6.0 	 8.0 	
 3.0 	 6.0 	 12.0 	 12.0 	
 4.0 	 8.0 	 12.0 	 20.0 	


In [40]:
M = Custom([1.4,π,√2,-4.5^5])

 3.3599999999999994 	 4.39822971502571 	 1.9798989873223332 	 -2583.3937499999997 	
 4.39822971502571 	 13.011197054679151 	 4.442882938158366 	 -5797.12201880699 	
 1.9798989873223332 	 4.442882938158366 	 3.414213562373096 	 -2609.621770142778 	
 -2583.3937499999997 	 -5797.12201880699 	 -2609.621770142778 	 3.4032176103515625e6 	


In [41]:
function f(λ, M::Custom)
    return 1 + sum([M.v[i]^2 / (M.v[i] - λ) for i in 1:size(M.v, 1)])
end

f (generic function with 1 method)

In [43]:
# Implementation of Newton's method to calculate the largest eigenvalue.

using ForwardDiff
using Printf

function max_eigenvalue(M::Custom; abs_tol=1e-6, max_iter=50)
    iter = 0
    λ = maximum(M.v) + 1  # Initial guess larger than maximum element as we
                          # know the largest eigenvalue is larger than it
    
    while abs(f(λ, M)) > abs_tol
        iter += 1
        λ = λ - ( f(λ, M) / ForwardDiff.derivative(λ -> f(λ, M), λ) )
        @printf("λ = %g, f(λ, M) = %g\n", λ, f(λ, M))
        iter ≥ max_iter && break
    end
    return λ
end

max_eigenvalue (generic function with 1 method)

In [49]:
# Looks correct by comparing with MATLAB's eig().
max_eigenvalue(Custom([1,2,3,4]))

λ = 6.12403, f(λ, M) = -10.5788
λ = 8.35493, f(λ, M) = -5.12008
λ = 12.3706, f(λ, M) = -2.34556
λ = 18.6125, f(λ, M) = -0.968977
λ = 26.0907, f(λ, M) = -0.31995
λ = 31.593, f(λ, M) = -0.0624721
λ = 33.2519, f(λ, M) = -0.00347175
λ = 33.3553, f(λ, M) = -1.19783e-05
λ = 33.3557, f(λ, M) = -1.43578e-10


33.35565805378168

In [50]:
# Looks correct by comparing with MATLAB's eig().
max_eigenvalue(Custom([-1/7, 4.44, π^2, 11, -π^3, 7]))

λ = 13.3799, f(λ, M) = -109.137
λ = 16.8772, f(λ, M) = -60.1131
λ = 26.0749, f(λ, M) = -33.3608
λ = 50.333, f(λ, M) = -17.8639
λ = 106.424, f(λ, M) = -8.95868
λ = 216.356, f(λ, M) = -4.2747
λ = 406.993, f(λ, M) = -1.91733
λ = 687.769, f(λ, M) = -0.760877
λ = 993.958, f(λ, M) = -0.229646
λ = 1183.53, f(λ, M) = -0.0361467
λ = 1225.56, f(λ, M) = -0.00121874
λ = 1227.07, f(λ, M) = -1.482e-06
λ = 1227.08, f(λ, M) = -2.19669e-12


1227.0752563677202

In [51]:
# I didn't use diagm and LinAlg as I can't get LinAlg to load in 1.0.

function sherman_morrison_inv(M::Custom)
    v = M.v
    N = size(v, 1)
    diag_v_inv = Array{Float64}(undef, N, N)
    for i = 1:N; diag_v_inv[i, i] = 1 / v[i]; end
    return diag_v_inv - (  (diag_v_inv * v * v' * diag_v_inv)
                         / (1 + v' * diag_v_inv * v))
end

sherman_morrison_inv (generic function with 1 method)

In [54]:
sherman_morrison_inv(Custom([1, 2, 3, 4]))
# Looks correct by comparing with MATLAB's inv().

4×4 Array{Float64,2}:
  0.909091   -0.0909091  -0.0909091  -0.0909091
 -0.0909091   0.409091   -0.0909091  -0.0909091
 -0.0909091  -0.0909091   0.242424   -0.0909091
 -0.0909091  -0.0909091  -0.0909091   0.159091 

In [55]:
sherman_morrison_inv(Custom([-1/7, 4.44, π^2, 11, -π^3, 7]))
# Looks correct by comparing with MATLAB's inv().

6×6 Array{Float64,2}:
 -7.46286   -0.462862  -0.462862  -0.462862  -0.462862  -0.462862
 -0.462862  -0.237637  -0.462862  -0.462862  -0.462862  -0.462862
 -0.462862  -0.462862  -0.361541  -0.462862  -0.462862  -0.462862
 -0.462862  -0.462862  -0.462862  -0.371953  -0.462862  -0.462862
 -0.462862  -0.462862  -0.462862  -0.462862  -0.495114  -0.462862
 -0.462862  -0.462862  -0.462862  -0.462862  -0.462862  -0.320005